In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define the path in your Google Drive where you want to save the model
MODEL_SAVE_PATH = "/content/drive/MyDrive/BRISC2025_Project/best_unet_model.pth"
CHECKPOINT_PATH = "/content/drive/MyDrive/BRISC2025_Project/latest_checkpoint.pth"
RESULTS_SAVE_PATH = "/content/drive/MyDrive/BRISC2025_Project/training_results.pth"

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import skimage
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from pathlib import Path

##Data preperations

In [ ]:
data_dir = "/content/BUS-UCLM Breast ultrasound lesion segmentation dataset/BUS-UCLM/images"
mask_dir = "/content/BUS-UCLM Breast ultrasound lesion segmentation dataset/BUS-UCLM/masks"
info = pd.read_csv("INFO.csv", sep = ';')

data_lst = os.listdir(data_dir)

In [1]:
# Install kagglehub
# !pip install kagglehub

import kagglehub

# Download latest version
path = kagglehub.dataset_download("briscdataset/brisc2025")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/brisc2025


###Path setup

In [ ]:
classification_task_train = os.path.join(path,"classification_task/train")
segmentation_task_train = os.path.join(path,"segmentation_task/train")
print(classification_task_train)
print(segmentation_task_train)

In [7]:
import os
from torch.utils.data import Dataset
from PIL import Image

class BrainSegmentationDataset(Dataset):
    def __init__(self, images_dir, masks_dir, transform=None):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform
        self.images = sorted(os.listdir(images_dir))
        self.masks = sorted(os.listdir(masks_dir))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.images_dir, self.images[idx])
        mask_path = os.path.join(self.masks_dir, self.masks[idx])

        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")  # grayscale mask

        if self.transform:
            # Apply the same transform to both
            augmented = self.transform(image=np.array(image), mask=np.array(mask))
            image, mask = augmented["image"], augmented["mask"]

        return image, mask

In [ ]:
img = BrainSegmentationDataset(classification_task, segmentation_task)